<a href="https://colab.research.google.com/github/slatej1/Everside/blob/main/468_Excel_Processor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

import numpy as np
import warnings
warnings.filterwarnings("ignore")
pd.options.display.max_columns = None
pd.options.display.max_rows = None


file_path =   'EFIV Q1 2024 SBIC Form 468 5.14.24.xlsx'
ironwood = pd.ExcelFile(file_path)
start_row = 19
df = pd.read_excel(ironwood, "S1Inv", skiprows=start_row)
new_header = df.iloc[0]
df = df[1:]
df.columns = new_header
df
df['Initial Financing Date'] = pd.to_datetime(df['Initial Financing Date'], format="%Y/%m/%d").dt.normalize()
df['First Investment Date'] = pd.to_datetime(df['First Investment Date'], format="%Y/%m/%d").dt.normalize()
dropped = ["Critical Technology\n(if applicable)", "Restructured?", "Class 1", "Class 2", "Class 2 Date of Up Round",
          "Prior SBA Mult"]

df = df.drop(dropped, axis=1)

In [ ]:
df

,Portfolio Company Name,Employer ID,Financing Type,Investment Type,Financing Description,Initial Financing Date,Equity Capital Investment,Interest or Dividend Rate,Ownership % (Fully Diluted),Loan/ Debt Status,Total Cash Invested (A),Cost at Beginning Period,Schedule 1C Reference Number,Addition/\nDeduction,Non-Cash Gain included in Cost at End of Period,Cost at End of Period,Unrealized Appreciation,(Unrealized Depreciation),SBA Reported Value (B),GAAP Reported Value (C),Cum. Cash Proceeds (D),Current SBA Mult,Current GAAP Mult,Prior GAAP Mult,First Investment Date
1,"RKD Group, LLC",47-5152075,Loan,Subordinated Debt,Investment concluded 1/30/2019,2018-05-01,N,NaN,NaN,Paid in Full,2703600,0,NaN,0,NaN,0,NaN,NaN,0,0,3026719.69,1.119515,1.119515,NaN,2018-05-01
2,"Vorteq Coil Finishers, LLC",46-4386937,Loan,Subordinated Debt,Investment concluded 11/30/2021,2018-05-01,N,NaN,NaN,Paid in Full,6045550,0,NaN,0,NaN,0,NaN,NaN,0,0,8550784.33,1.414393,1.414393,NaN,2018-05-01
3,"GAHH, LLC",20-3501888,Loan,Subordinated Debt,Investment concluded 2/25/2022,2018-05-09,N,NaN,NaN,Paid in Full,4581100,0,NaN,0,NaN,0,NaN,NaN,0,0,6364731.39,1.389346,1.389346,NaN,2018-05-09
4,"GAHH, LLC",20-3501888,Equity,Preferred Stock (Equity),600.8 Shares of Series A Preferred Shares; Inv...,2018-05-09,Y,NaN,NaN,NaN,600800,0,NaN,0,NaN,0,NaN,NaN,0,0,901592.83,1.500654,1.500654,NaN,2018-05-09
5,"GAHH, LLC",20-3501888,Equity,Common Stock (Equity),60.08 Voting Common Shares; Investment conclud...,2018-05-09,Y,NaN,NaN,NaN,600.8,0,NaN,0,NaN,0,NaN,NaN,0,0,1702741,2834.122836,2834.122836,NaN,2018-05-09
6,"VPI Acquisition, LLC",20-4960742,Loan,Subordinated Debt,Investment concluded 6/2/2023,2018-06-20,N,NaN,NaN,Paid in Full,2883592,0,NaN,0,NaN,0,NaN,NaN,0,0,4895837.87,1.697826,1.697826,NaN,2018-06-20
7,"Boutique Brands, LLC",83-0687802,Loan,Subordinated Debt,Current interest due quarterly except during d...,2018-07-12,N,0.14,NaN,Performing,5915758.33,7879032.47,16,278830.08,NaN,8157862.55,NaN,-113100,8044762.55,0,1961806.16,1.691511,0.331624,NaN,2018-07-12
8,"Boutique Brands, LLC",83-0687802,Equity,Preferred Stock (Equity),"1,281,862 Class A Preferred Shares and 1,775,4...",2018-07-12,Y,NaN,0.0966,NaN,1725720.22,1725720.22,NaN,0,NaN,1725720.22,NaN,NaN,1725720.22,0,327732.37,1.18991,0.18991,NaN,2018-07-12
9,"OJ Medtech, Inc.",58-2429957,Loan,Subordinated Debt,"Current interest due quarterly, except during ...",2018-07-18,N,0.13,NaN,Delinquent/Default,4377600,5386113.5,NaN,0,NaN,5386113.5,NaN,-115200,5270913.5,0,1942934.54,1.6479,0.443836,NaN,2018-07-18
10,"OJ Medtech, Inc.",58-2429957,Equity,Preferred Stock (Equity),17.4 shares of Class A preferred stock,2018-07-18,Y,NaN,0.0361,NaN,998400,998400,NaN,0,NaN,998400,NaN,-517524,480876,0,49920.0,0.531647,0.05,NaN,2018-07-18


In [ ]:
loan = ['Loan', 'Debt']
# Separate equity and loan types
warrants = ["Warrants (Debt)", "Warrants (Equity)"]
equity_df = df[(df['Financing Type'] == 'Equity') | (df["Investment Type"].isin(warrants))]
loan_df = df[(df['Financing Type'].isin(loan)) & (~df["Investment Type"].isin(warrants))]

In [ ]:
pattern = r'Maturity Date: (\d{2}/\d{2}/\d{4})'
debt_status = {"Paid in Full": "Exited", "Performing":"Performing", "Delinquent/Default": "Exited", "Charge Off": "Exited", "Covenant Issues": "Under Expectation", "Other Concerns": "Under Expectation", "Foreberance":"Under Expectation"}


loan_df["Debt Pricing - Maturity"] = loan_df["Financing Description"].str.extract(pattern)
loan_df["Status Compliance of Credit - Current Status"] = loan_df["Loan/ Debt Status"].map(debt_status)

In [ ]:
loan_df

,Portfolio Company Name,Employer ID,Financing Type,Investment Type,Financing Description,Initial Financing Date,Equity Capital Investment,Interest or Dividend Rate,Ownership % (Fully Diluted),Loan/ Debt Status,Total Cash Invested (A),Cost at Beginning Period,Schedule 1C Reference Number,Addition/\nDeduction,Non-Cash Gain included in Cost at End of Period,Cost at End of Period,Unrealized Appreciation,(Unrealized Depreciation),SBA Reported Value (B),GAAP Reported Value (C),Cum. Cash Proceeds (D),Current SBA Mult,Current GAAP Mult,Prior GAAP Mult,First Investment Date,Maturity Date,Debt Pricing - Maturity,Status Compliance of Credit - Current Status
1,"RKD Group, LLC",47-5152075,Loan,Subordinated Debt,Investment concluded 1/30/2019,2018-05-01,N,NaN,NaN,Paid in Full,2703600,0,NaN,0,NaN,0,NaN,NaN,0,0,3026719.69,1.119515,1.119515,NaN,2018-05-01,NaN,NaN,Exited
2,"Vorteq Coil Finishers, LLC",46-4386937,Loan,Subordinated Debt,Investment concluded 11/30/2021,2018-05-01,N,NaN,NaN,Paid in Full,6045550,0,NaN,0,NaN,0,NaN,NaN,0,0,8550784.33,1.414393,1.414393,NaN,2018-05-01,NaN,NaN,Exited
3,"GAHH, LLC",20-3501888,Loan,Subordinated Debt,Investment concluded 2/25/2022,2018-05-09,N,NaN,NaN,Paid in Full,4581100,0,NaN,0,NaN,0,NaN,NaN,0,0,6364731.39,1.389346,1.389346,NaN,2018-05-09,NaN,NaN,Exited
6,"VPI Acquisition, LLC",20-4960742,Loan,Subordinated Debt,Investment concluded 6/2/2023,2018-06-20,N,NaN,NaN,Paid in Full,2883592,0,NaN,0,NaN,0,NaN,NaN,0,0,4895837.87,1.697826,1.697826,NaN,2018-06-20,NaN,NaN,Exited
7,"Boutique Brands, LLC",83-0687802,Loan,Subordinated Debt,Current interest due quarterly except during d...,2018-07-12,N,0.14,NaN,Performing,5915758.33,7879032.47,16,278830.08,NaN,8157862.55,NaN,-113100,8044762.55,0,1961806.16,1.691511,0.331624,NaN,2018-07-12,02/12/2025,02/12/2025,Performing
9,"OJ Medtech, Inc.",58-2429957,Loan,Subordinated Debt,"Current interest due quarterly, except during ...",2018-07-18,N,0.13,NaN,Delinquent/Default,4377600,5386113.5,NaN,0,NaN,5386113.5,NaN,-115200,5270913.5,0,1942934.54,1.6479,0.443836,NaN,2018-07-18,09/18/2024,09/18/2024,Exited
12,ITRAC LLC dba Dental Revolution,83-1367490,Loan,Subordinated Debt,Current interest due monthly. Principal and PI...,2018-08-01,N,0.115,NaN,Performing,6528000,6936820.86,17,26302.11,NaN,6963122.97,NaN,NaN,6963122.97,0,2971646.19,1.52187,0.455215,NaN,2018-08-01,08/01/2025,08/01/2025,Performing
14,"Paragon Bioservices, Inc.",52-1667981,Loan,Subordinated Debt,Investment concluded 5/17/2019,2018-08-03,N,NaN,NaN,Paid in Full,5760000,0,NaN,0,NaN,0,NaN,NaN,0,0,6686400,1.160833,1.160833,NaN,2018-08-03,NaN,NaN,Exited
16,ENT&C Management LLC,83-1360831,Loan,Subordinated Debt,"Current interest due monthly, except during de...",2018-09-13,N,0.14,NaN,Performing,6612503.1,7251317.83,12,36721.21,NaN,7288039.04,NaN,NaN,7288039.04,0,3577769.93,1.643222,0.541061,NaN,2018-09-13,06/30/2025,06/30/2025,Performing
18,"Coronis Health, LLC",47-5481090,Loan,Subordinated Debt,Investment concluded 7/12/2022,2018-11-30,N,NaN,NaN,Paid in Full,9681196,0,NaN,0,NaN,0,NaN,NaN,0,0,14944141.98,1.543626,1.543626,NaN,2018-11-30,NaN,NaN,Exited


In [ ]:
def aggregate_values(group):
    result = group.iloc[0].copy()
    result['Total Cash Invested (A)'] = group['Total Cash Invested (A)'].sum()
    result['Cost at Beginning Period'] = group['Cost at Beginning Period'].sum()
    result['Non-Cash Gain included in Cost at End of Period'] = group['Non-Cash Gain included in Cost at End of Period'].sum()
    result['Unrealized Appreciation'] = group['Unrealized Appreciation'].sum()
    result['Cost at End of Period'] = group['Cost at End of Period'].sum()
    result['(Unrealized Depreciation)'] = group['(Unrealized Depreciation)'].sum()
    result['SBA Reported Value (B)'] = group['SBA Reported Value (B)'].sum()
    result['GAAP Reported Value (C)'] = group['GAAP Reported Value (C)'].sum()
    result['Cum. Cash Proceeds (D)'] = group['Cum. Cash Proceeds (D)'].sum()
    result['Initial Financing Date'] = group['Initial Financing Date'].min()
    result['First Investment Date'] = group['First Investment Date'].min()
    result['Addition/\nDeduction'] = group['Addition/\nDeduction'].sum()

    ownership = group['Ownership % (Fully Diluted)']
    if ownership.isna().all():
        result['Ownership % (Fully Diluted)'] = np.nan
    else:
        result['Ownership % (Fully Diluted)'] = ownership.fillna(0).sum()

    investment_types = group['Investment Type']
    non_other_types = investment_types[investment_types != 'Other'].dropna().unique()
    if len(non_other_types) > 0:
        result['Investment Type'] = non_other_types[0]
    else:
        result['Investment Type'] = 'Other'

    return result


equity_df = equity_df.groupby('Portfolio Company Name').apply(aggregate_values).reset_index(drop=True)
loan_df = loan_df.groupby('Portfolio Company Name').apply(aggregate_values).reset_index(drop=True)

In [ ]:
loan_df.columns = ['Company Name', 'Employer ID', 'Financing Type',
       'Investment Type', 'Financing Description', 'Initial Financing Date',
       'Equity Capital Investment', 'Debt Pricing - Total Coupon',
       'Ownership % (Fully Diluted)', 'Loan/ Debt Status',
       'Invested Dollars - Debt Cost', 'Cost at Beginning Period',
       'Schedule 1C Reference Number', 'Addition/\nDeduction',
       'Non-Cash Gain included in Cost at End of Period',
       'Cost at End of Period', 'Unrealized Appreciation',
       '(Unrealized Depreciation)', 'SBA Reported Value (B)',
      'Invested Dollars - Debt FMV', 'Cum. Cash Proceeds (D)', 'Current SBA Mult',
       'Current GAAP Mult', 'Prior GAAP Mult', 'First Investment Date']

equity_df.columns =['Company Name', 'Employer ID', 'Financing Type',
       'Investment Type', 'Financing Description', 'Initial Financing Date',
       'Equity Capital Investment', 'Interest or Dividend Rate',
       'Equity Pricing - Equity Ownership (FD)', 'Loan/ Debt Status',
        'Invested Dollars - Equity Cost', 'Cost at Beginning Period',
       'Schedule 1C Reference Number', 'Addition/\nDeduction',
       'Non-Cash Gain included in Cost at End of Period',
       'Cost at End of Period', 'Unrealized Appreciation',
       '(Unrealized Depreciation)', 'SBA Reported Value (B)',
       'Invested Dollars - Equity FMV', 'Cum. Cash Proceeds (D)', 'Current SBA Mult',
       'Current GAAP Mult', 'Prior GAAP Mult', 'First Investment Date']
ec , lc = [' Company Name','Equity Pricing - Equity Ownership (FD)',  'Invested Dollars - Equity Cost', 'Invested Dollars - Equity FMV'],['Company Name','Debt Pricing - Total Coupon', 'Invested Dollars - Debt Cost', 'Invested Dollars - Debt FMV']

In [ ]:
template_file_path ="Updated Quarterly Reporting Template 1.xlsx"
new_file_path = 'EFv6.xlsx'


# Load the template Excel file
excel_df = pd.read_excel(template_file_path)

# Load the DataFrame you want to use for updating the template
# Assuming df is already defined and loaded

# Iterate over the columns in the Excel file and update with values from df
for col in excel_df.columns:
    if col in ec:
        excel_df[col] = equity_df[col]
    elif col in lc:
      excel_df[col] = loan_df[col]
    else:
      continue


# Save the updated DataFrame to a new file
excel_df.to_excel(new_file_path, index=False)

print("Excel file successfully populated and saved as", new_file_path)

Excel file successfully populated and saved as EFv6.xlsx


In [ ]:
number_of_companies = len(set(equity_df['Employer ID'] + loan_df['Employer ID'] ))
print(number_of_companies)


22


In [ ]:
source_file_path =  "03-31-24 Plexus Fund IV-A - 468 (1).xlsx"
data = pd.read_excel(source_file_path, sheet_name="S12pc")

def collect_information(data, offset):
    dict = {"Name of License":data.iloc[offset + 2,1 ],
            "Portfolio Company": data.iloc[offset + 3, 2 ],
            "1st Date Invested": data.iloc[offset + 4, 2],
            "NAICS": data.iloc[offset + 5, 2],
            "HQ City": data.iloc[offset + 6, 2],
             "HQ State": data.iloc[offset + 6, 5],
            "EV at 1st Closing": data.iloc[offset + 7, 2],
             "SBA Reported Valued": data.iloc[offset + 9, 2],
            "Fund Own%": data.iloc[offset + 4, 8],
            "Associate Own%	": data.iloc[offset + 5, 8],
            "Management Own%": data.iloc[offset + 6, 8],
            "Board Representation": data.iloc[offset + 7, 8],
            "Vote%": data.iloc[offset + 9, 8],
            "Invest. Allocation": data.iloc[offset + 4, 11],
            "Invested Capital": data.iloc[offset + 5, 11],
             "Realized Proceeds": data.iloc[offset + 6, 11],
            "GAAP Value": data.iloc[offset + 7, 11],
            "Investment Multiple": data.iloc[offset + 8, 11],
            "Gross IRR": data.iloc[offset + 9, 11],

            "As of Date":[data.iloc[offset + 19,2 ],data.iloc[offset + 19,3 ],data.iloc[offset + 19,4 ], data.iloc[offset + 19,5 ] ],
             "Sales":[data.iloc[offset + 20,2 ],data.iloc[offset + 20,3 ],data.iloc[offset + 20,4 ],data.iloc[offset + 20,5 ]  ],
            "YOY % Growth":[data.iloc[offset + 21,2 ],data.iloc[offset + 21,3 ],data.iloc[offset + 21,4 ], data.iloc[offset + 21,5 ]  ],
            "EBITDA":[data.iloc[offset + 22,2 ],data.iloc[offset + 22,3 ],data.iloc[offset + 22,4 ], data.iloc[offset + 22,5 ]  ],
            "EBITDA Margin": [data.iloc[offset + 24, 2], data.iloc[offset + 24, 3],  data.iloc[offset + 24, 4],  data.iloc[offset + 24, 5]],
             "Enterprise Value":[data.iloc[offset + 19,8 ],data.iloc[offset + 19,9 ],data.iloc[offset + 19,10 ],data.iloc[offset + 19,11 ] ],
          "TEV Multiple":[data.iloc[offset + 20,8 ],data.iloc[offset + 20,9 ],data.iloc[offset + 20,10 ],data.iloc[offset + 20,11 ]  ],
             "Total Debt":[data.iloc[offset + 21,8 ],data.iloc[offset + 21,9 ],data.iloc[offset + 21,10 ],data.iloc[offset + 21,11 ]  ],
    "Total Lev Multiple":[data.iloc[offset + 22,8 ],data.iloc[offset + 22,9 ],data.iloc[offset + 22,10 ],data.iloc[offset + 22,11 ]  ],
    "Liquidity (Cash + Available Liquidity)": [data.iloc[offset + 23, 8], data.iloc[offset + 23, 9],  data.iloc[offset + 23, 10],  data.iloc[offset + 24, 11]],

            "At Close Sales": data.iloc[offset+ 20,2], "Current Sales":data.iloc[offset + 20,5 ], "At Close EBITDA":data.iloc[offset + 22,2 ], "Current EBITDA":data.iloc[offset + 22,5 ],
            "At Close Enterprise Value":data.iloc[offset + 19,8 ], "At Close Total Debt":data.iloc[offset + 21,8 ], "At Close Liquidity (Cash + Available Liquidity)":data.iloc[offset + 23, 8],
        "Current Total Debt":data.iloc[offset + 21,11 ], "Current Enterprise Value":data.iloc[offset + 19,11 ]
    }

    return dict

In [ ]:
#Takes relevant information and writes into the output of 468_General.py
template_file_path = 'EFv6.xlsx'
new_file_path = 'Plexus-468.xlsx'

# Load the template Excel file
excel_df = pd.read_excel(template_file_path)

cols = ["At Close Sales", "Current Sales", "At Close EBITDA", "Current EBITDA", "At Close Enterprise Value", "At Close Total Debt",
        "Current Total Debt", "Current Enterprise Value"]

# Iterate over the columns in the Excel file and update with values from df
offset=0
for i in range(number_of_companies):
    company = collect_information(data, offset)
    offset+=47
    company_row = excel_df[excel_df['Company Name'] == company["Portfolio Company"]]

    # Update the values in the row
    for key, value in company.items():
        if key in cols and not company_row.empty:
            excel_df.loc[company_row.index, key] = value

# Save the updated DataFrame to a new file
excel_df.to_excel(new_file_path, index=False)

print("Excel file successfully populated and saved as", new_file_path)


Excel file successfully populated and saved as Plexus-468.xlsx
